# Introduction to the data

In [5]:
import pandas as pd
pd.options.mode.chained_assignment = None
academy_awards = pd.read_csv("academy_awards.csv",encoding="ISO-8859-1")

# Filtering The Data

In [6]:
academy_awards["Year"] = academy_awards["Year"].str[0:4]
academy_awards["Year"] = academy_awards["Year"].astype("int64")
later_than_2000 = academy_awards[academy_awards["Year"] > 2000]
award_categories = ["Actor -- Leading Role","Actor -- Supporting Role","Actress -- Leading Role", "Actress -- Supporting Role"]

nominations = later_than_2000[later_than_2000["Category"].isin(award_categories)]


# Cleaning up the Won? and Unnamed columns

In [7]:
replacements = { "NO": 0, "YES": 1 }
nominations["Won"] = nominations["Won?"].map(replacements)
drop_cols = ["Won?","Unnamed: 5", "Unnamed: 6","Unnamed: 7", "Unnamed: 8", "Unnamed: 9", "Unnamed: 10"]
final_nominations = nominations.drop(drop_cols, axis=1)

# Cleaning up the Additional Info colum

In [8]:
additional_info_one = final_nominations["Additional Info"].str.rstrip("'}")
additional_info_two = additional_info_one.str.split(" {'")
movie_names = additional_info_two.str[0]
characters = additional_info_two.str[1]
final_nominations["Movie"] = movie_names
final_nominations["Character"] = characters
final_nominations = final_nominations.drop("Additional Info",axis=1)
final_nominations.head()

,Year,Category,Nominee,Won,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,0,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,0,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,0,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,1,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,0,127 Hours,Aron Ralston


# Exporting to SQLite

In [11]:
import sqlite3
conn = sqlite3.connect("nominations1.db")
final_nominations.to_sql("nominations1",conn,index=False)

# Verifying in SQL

In [12]:
query_one = "pragma table_info(nominations1);"
query_two = "select * from nominations1 limit 10;"
print(conn.execute(query_one).fetchall())
print(conn.execute(query_two).fetchall())
conn.close()


[(0, 'Year', 'INTEGER', 0, None, 0), (1, 'Category', 'TEXT', 0, None, 0), (2, 'Nominee', 'TEXT', 0, None, 0), (3, 'Won', 'INTEGER', 0, None, 0), (4, 'Movie', 'TEXT', 0, None, 0), (5, 'Character', 'TEXT', 0, None, 0)]
[(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal'), (2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn'), (2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg'), (2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI'), (2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston'), (2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund'), (2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop'), (2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin'), (2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul'), (2010, 'Act